In [2]:
import os
os.chdir('C:/Users/myhyu/OneDrive/Desktop/2024 미래에셋/InvestmentPropensity')

In [3]:
# 라이브러리 
import requests
import pandas as pd
import io
import time
import requests
from bs4 import BeautifulSoup
import numpy as np

In [4]:
def get_ev_ebitda(stock_code):
    # URL 설정
    url = f'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{stock_code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'

    # HTTP 요청 및 페이지 내용 가져오기
    response = requests.get(url)
    response.raise_for_status()  # 요청 실패 시 예외 발생

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # 포함하는 행
    table = soup.find_all('tr', {'class': 'c_grid1_14 rwf acd_dep2_sub'})
    
    # 추출
    try:
        ev_value = table[0].find('td', {'class': 'cle r tdbg_y'}).text
        ev = pd.to_numeric(ev_value.replace(',', ''), errors='coerce')
    except (IndexError, AttributeError):
        ev = pd.NA

    try:
        ebitda_value = table[1].find('td', {'class': 'cle r tdbg_y'}).text
        ebitda = pd.to_numeric(ebitda_value.replace(',', ''), errors='coerce')
    except (IndexError, AttributeError):
        ebitda = pd.NA

    if pd.notna(ev) and pd.notna(ebitda) and ebitda != 0:
        ev_ebitda = ev / ebitda
    else:
        ev_ebitda = pd.NA

    # 데이터프레임 생성
    data = {
        '기업코드': [stock_code],
        'EV': [ev],
        'EBITDA': [ebitda],
        'EV/EBITDA' : [ev_ebitda]
    }
    df = pd.DataFrame(data)
    return df

In [5]:
# 저장된 종목코드 불러오기
code_data = pd.read_csv('code.csv')
code_data = code_data[['종목코드']]
code_data

,종목코드
0,095570
1,006840
2,027410
3,282330
4,138930
...,...
2691,024060
2692,010240
2693,189980
2694,037440


In [6]:
# 재무제표 크롤링
for idx, firm_code in enumerate(code_data['종목코드']):
    print(idx + 1, '/', len(code_data['종목코드']))
    try:
        # 종목별 재무제표 크롤링
        try:
            sub_finance_table = get_ev_ebitda(firm_code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            sub_finance_table = get_ev_ebitda(firm_code)

        # 종목별 재무제표 정리
        for num, col in enumerate(sub_finance_table.columns):
            temp = pd.DataFrame({firm_code: sub_finance_table[col]}).T
            temp.columns = [[col] * len(sub_finance_table), temp.columns]
            if num == 0:
                temp_arranged = temp
            else:
                temp_arranged = pd.merge(temp_arranged, temp, how='outer', left_index=True, right_index=True)

        # 재무제표 통합        
        if idx == 0:
            finance_table = temp_arranged
        else:
            finance_table = pd.concat([finance_table, temp_arranged])
    except (ValueError, KeyError):
        continue


1 / 2696
2 / 2696
3 / 2696
4 / 2696
5 / 2696


C:\Users\myhyu\AppData\Local\Temp\ipykernel_11084\1729452186.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finance_table = pd.concat([finance_table, temp_arranged])


6 / 2696
7 / 2696


C:\Users\myhyu\AppData\Local\Temp\ipykernel_11084\1729452186.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finance_table = pd.concat([finance_table, temp_arranged])


8 / 2696
9 / 2696
10 / 2696


...

2648 / 2696
2649 / 2696


C:\Users\myhyu\AppData\Local\Temp\ipykernel_11084\1729452186.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finance_table = pd.concat([finance_table, temp_arranged])


2650 / 2696


C:\Users\myhyu\AppData\Local\Temp\ipykernel_11084\1729452186.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finance_table = pd.concat([finance_table, temp_arranged])


2651 / 2696
2652 / 2696
2653 / 2696
2654 / 2696
2655 / 2696
2656 / 2696
2657 / 2696
2658 / 2696
2659 / 2696
2660 / 2696
2661 / 2696
2662 / 2696
2663 / 2696
2664 / 2696
2665 / 2696
2666 / 2696
2667 / 2696
2668 / 2696
2669 / 2696
2670 / 2696
2671 / 2696
2672 / 2696
2673 / 2696
2674 / 2696
2675 / 2696
2676 / 2696
2677 / 2696
2678 / 2696
2679 / 2696
2680 / 2696
2681 / 2696
2682 / 2696
2683 / 2696
2684 / 2696
2685 / 2696
2686 / 2696
2687 / 2696
2688 / 2696
2689 / 2696
2690 / 2696
2691 / 2696
2692 / 2696
2693 / 2696
2694 / 2696
2695 / 2696
2696 / 2696


In [7]:
finance_table.info()
# 결측치 300개

<class 'pandas.core.frame.DataFrame'>
Index: 2696 entries, 095570 to 238490
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (기업코드, 0)       2696 non-null   object 
 1   (EV, 0)         2400 non-null   object 
 2   (EBITDA, 0)     2401 non-null   object 
 3   (EV/EBITDA, 0)  2379 non-null   float64
dtypes: float64(1), object(3)
memory usage: 105.3+ KB


In [11]:
finance_table.columns

MultiIndex([(     '기업코드', 0),
            (       'EV', 0),
            (   'EBITDA', 0),
            ('EV/EBITDA', 0)],
           )

In [16]:
finance_table.columns = ['기업코드','EV','EBITDA','EV/EBITDA']
finance_table

,기업코드,EV,EBITDA,EV/EBITDA
095570,095570,11528,615,18.744715
006840,006840,27192,1606,16.931507
027410,027410,6049,185,32.697297
282330,282330,20343,1597,12.738259
138930,138930,<NA>,<NA>,NaN
...,...,...,...,...
024060,024060,1572,-2,-786.000000
010240,010240,690,40,17.250000
189980,189980,1725,37,46.621622
037440,037440,947,32,29.593750


In [18]:
# 재무제표 저장
finance_table.to_csv('ev_ebitda_table.csv', index=False, encoding='cp949')